In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import string
import time

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re

In [2]:
#input file 
df=pd.read_excel("universitylinks.xlsx")

#Stopwords
stop_words = stopwords.words('english')

#positive-negative
positive_words=pd.read_csv("positive-words.txt",header=None)[0].tolist()
negative_words=pd.read_csv("negative-words.txt",header=None, encoding="ISO-8859-1")[0].tolist()

In [3]:
options=Options()
options.headless=True
options.add_argument('window-size=1366x768')

In [4]:
tokenizer = RegexpTokenizer('\w+[a-zA-Z\-]*')
punctuations = string.punctuation
punctuation_list=[*punctuations]

In [5]:
links=df['University_Links']

URLs=[]
positive_score_list=[]
negative_score_list=[]
polarity_score_list=[]
subjective_score_list=[]
avg_sent_len_list=[]
percent_complex_word_list=[]
Fog_index_list=[]
avg_num_of_words_per_sent_list=[]
complex_word_count_list=[]
word_count_list=[]
syllable_per_word_list=[]
personal_pronoun_list=[]
avg_word_length_list=[]




In [6]:
def positivescore(result,positive_words):
    counter=0
    for ele in result:
        if ele in positive_words:
            counter=counter+1
    return counter

In [7]:
def negativescore(result,negative_words):
    counter=0
    for ele in result:
        if ele in negative_words:
            counter=counter+1
    return counter
    

In [8]:
def averagesentlen(result,res_without_puntuations):
    sent_tokens = nltk.sent_tokenize(result)
    avg_sent_len=len(res_without_puntuations)/len(sent_tokens)
    return avg_sent_len

In [9]:
def word_count(result,stop_words):
    num_of_words=0
    for ele in result:
        if ele not in stop_words:
                num_of_words=num_of_words+1
    return num_of_words

In [10]:
def complexword(result,stop_words):
    words=[]
    for ele in result:
        if ele not in stop_words:
            words.append(ele)
    Complex_count=0
    for word in words:
        SyllablePerWord=0
        for w in word:
            if (w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'):
                SyllablePerWord+=1
        if word.endswith("ed" or "es"):
            SyllablePerWord-=1
        if SyllablePerWord>=3:
            Complex_count+=1
    return Complex_count

In [11]:
def Syllable(result,stop_words):
    words=[]
    for ele in result:
        if ele not in stop_words:
            words.append(ele)
    Syllable_count=0
    for word in words:
        for w in word:
            if (w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'):
                Syllable_count+=1
        if word.endswith("ed" or "es"):
            Syllable_count-=1
    return Syllable_count

In [12]:
def pronoun(result):
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronouns = pronounRegex.findall(result)
    return len(pronouns)

#reference  link: https://stackoverflow.com/questions/70691981/finding-the-number-of-personal-pronouns-used-in-text-using-regrex-with-mixed-cas

In [13]:
def average_word_length(res_without_puntuations):
    letters=0
    for word in res_without_puntuations:
        for w in word:
            letters=letters+1
    avg=letters/len(res_without_puntuations)
    return avg

In [14]:
for link in links:
    path = Service('C:/Users/krush/Downloads/chromedriver_win32/chromedriver.exe')
    driver = webdriver.Chrome(service=path, options=options)
    driver.get(link)
    driver.maximize_window()
    
    try:
    
        title=driver.find_element(By.XPATH,"//h1[@class='text-white']").text
        content_list=[]


        more_link = driver.find_element(By.XPATH,'//*[@id="p2-overview"]/div[3]/div/div/div[1]/p/span/a')
        more_link.click()
        time.sleep(2)

        cont=driver.find_elements(By.XPATH,'//*[@id="p2-overview"]/div[3]/div/div/div[2]/p')
        for i in cont:
                content_list.append(i.text)


        delim = ""
        textoutput =  delim.join(content_list)

        URLs.append(link)

        result = title + " " + textoutput

        title=title.replace('?','')
        title=title.replace(':','')
        with open(f'Text-outputs/{title}.txt','w+',encoding='utf-8') as file:
            file.write(str(result))


        res_without_puntuations=tokenizer.tokenize(result)
        res_without_stopwords=[]
        for ele in res_without_puntuations:
            if ele not in stop_words:
                res_without_stopwords.append(ele)

        #positive score
        positive_score=positivescore(res_without_stopwords,positive_words)
        positive_score_list.append(positive_score)

        #negative score
        negative_score=negativescore(res_without_stopwords,negative_words)
        negative_score_list.append(negative_score)

        #polarity score
        polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
        polarity_score_list.append(polarity_score)

        #subjective score
        subjective_score=(positive_score + negative_score)/ (len(res_without_stopwords) + 0.000001)
        subjective_score_list.append(subjective_score)

        #Average Sentence Length
        average_sent_length=averagesentlen(result,res_without_puntuations)
        avg_sent_len_list.append(average_sent_length)


        #Average Number of Words Per Sentence = the total number of words / the total number of sentences
        avg_num_of_words_per_sent=average_sent_length
        avg_num_of_words_per_sent_list.append(avg_num_of_words_per_sent)

        #Complex Word Count
        complex_word_count=complexword(res_without_puntuations,stop_words)
        complex_word_count_list.append(complex_word_count)

        #Word Count
        number_of_words = word_count(res_without_puntuations,stop_words)
        word_count_list.append(number_of_words)

        #Percentage of Complex words  = the number of complex words / the number of words
        percent_complex_word=100*complex_word_count/number_of_words
        percent_complex_word_list.append(percent_complex_word)

        #Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
        Fog_index=0.4*(average_sent_length + percent_complex_word)
        Fog_index_list.append(Fog_index)


        #Syllable Count Per Word      
        Syallable_count=Syllable(res_without_puntuations,stop_words)
        Syllable_per_word=Syallable_count/number_of_words
        syllable_per_word_list.append(Syllable_per_word)


        #Personal Pronouns.
        personal_pronoun=pronoun(result)
        personal_pronoun_list.append(personal_pronoun)

        #Average Word Length
        avg_word_length=average_word_length(res_without_puntuations)
        avg_word_length_list.append(avg_word_length)
    except:
        pass
    

In [20]:
dataframe=pd.DataFrame({'URL':URLs, 'POSITIVE SCORE':positive_score_list, 'NEGATIVE SCORE':negative_score_list, 'POLARITY SCORE':polarity_score_list, 'SUBJECTIVE SCORE':subjective_score_list, 'AVG SENTENCE LENGTH':avg_sent_len_list, 'PERCENTAGE OF COMPLEX WORDS':percent_complex_word_list, 'FOG INDEX':Fog_index_list, 'AVG NUMBER OF WORDS PER SENTENCE':avg_num_of_words_per_sent_list, 'COMPLEX WORD COUNT':complex_word_count_list, 'WORD COUNT':word_count_list, 'SYLLABLE PER WORD':syllable_per_word_list, 'PERSONAL PRONOUNS':personal_pronoun_list, 'AVG WORD LENGTH':avg_word_length_list})

In [16]:
dataframe.shape

(82, 14)

In [21]:
dataframe.head()

,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVE SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,https://www.topuniversities.com/universities/m...,14,2,0.750000,0.049231,26.947368,42.461538,27.763563,26.947368,138,325,2.415385,0,5.261719
1,https://www.topuniversities.com/universities/u...,12,2,0.714286,0.043210,31.529412,44.444444,30.389542,31.529412,144,324,2.549383,0,5.341418
2,https://www.topuniversities.com/universities/s...,15,2,0.764706,0.058824,27.176471,38.062284,26.095502,27.176471,110,289,2.429066,0,5.303030
3,https://www.topuniversities.com/universities/u...,17,2,0.789474,0.059190,26.600000,37.383178,25.593271,26.600000,120,321,2.376947,1,5.082707
4,https://www.topuniversities.com/universities/h...,14,2,0.750000,0.054237,29.312500,37.627119,26.775847,29.312500,111,295,2.369492,0,5.198294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,https://www.topuniversities.com/universities/p...,2,0,1.000000,0.012821,32.500000,52.564103,34.025641,32.500000,82,156,2.724359,0,5.507692
78,https://www.topuniversities.com/universities/u...,22,0,1.000000,0.055138,26.913043,50.375940,30.915593,26.913043,201,399,2.634085,0,5.428110
79,https://www.topuniversities.com/universities/l...,15,0,1.000000,0.046729,32.533333,46.728972,31.704922,32.533333,150,321,2.570093,0,5.635246
80,https://www.topuniversities.com/universities/u...,28,2,0.866667,0.171429,21.384615,49.142857,28.210989,21.384615,86,175,2.657143,0,5.464029


In [18]:
dataframe.to_excel('output.xlsx')

In [19]:
driver.quit()